# QUESTION 2 #
## Simulation / Operations Research appoach ##


The same input set of data for tagging stations with a utility measure may feed a simulation
model to turn up it a notch on data and proceed forward on a richer model for decision making.
Simulation will sample the space of events of 'free_bikes' instantiation for every station and apply
a route problem solving algorithm. This way the dynamic of the system may be characterized to discover 
the delays and cost for choices in day to day routing. The final result will converge to a 
space map containing a grid with metrics expressing the mean time of commuting from home to Catedral Bicing Station.

The model comprises a more or less dense grid of spots spanning across a patch inside Gracia district.
The simulation will consider each spot a potencial home and hence the point of departure for a routing
problem. For every spot the simulation will play E times the game of "what happens if...?" throwing
a dice for every station and giving a number of bikes following a probability distribution. Once stations
have "free_bikes" assigned (a random value from 0 to some MAX), an algorithm will emulate human decission
process chosing a set of steps from station to station until getting a bike to move to Catedral Station.
The simulation will retrieve metrics from every "game" and produce a final metric of "mean commute time"
for every spot in the grid. The choice of a most promising area/street to settle is bound to chose the most
scored spot in the grid.



In [23]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

'''
QUESTION 2 for HOSCO Data Engineer assignment; Operations Research-Simulation Approach.

    Find most favourable bike-stations to choose a place for living in Gracia district.
    Operations Research approach will allow an extra level of metric extraction :
    the Mean Commute Time.


'''

import requests
import urllib, json

import datetime

import pymongo as mongo
from pymongo import MongoClient

import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt
import random as rnd

import bson

import sys
sys.path.insert(0, './gpsDistance')
sys.path.insert(0, './graphPath')

import DAGShortestWeightedEdgePath as DAGShortestWeightedEdgePath  
import gpsDistance as gpsDistance

import copy

__author__ = "Alexis Torrano"
__email__ = "a.torrano.m@gmail.com"
__status__ = "Production"

%matplotlib inline


## Imports and code reusing ##
Both versions of scripts for answering Question 2 share many methods and pieces of code. Indeed, the simulation version is an extension to the first approach. But as it is more comfortable (IMHO) to have and see all code in the same notebook no 'import' was used pointing to the first version. 

**def countZeroBikeStations(snapshot,tablCounter):**

Count for every station the amount of 5 minute segments with no available bikes.

**def getStaticFeaturesOfStations(snapshot,volatileFeatures):**

Extract the non changing features for each station across the time.

**def getHeatSpots(pandasDF,metrics,m,inverseHeat,heatSpotsList):**
    
Get the points which induce the clouds of heat around each station. The cloud of spots depends on the
amount of time segments where the station was depleted.
        
**def heatmap(pandasGraciaStations,pandasNeighbours,metrics,inverseHeat=False):**
    
Produces a map object where the heatmap layer was added.

In [17]:
def countZeroBikeStations(snapshot,tablCounter):
    GraciaStations = snapshot['gracia']
    neighbourStations = snapshot['neighbours']    
    
    # Increase the counter of each station when bikes are depleted.
    # If station is stil not registered in the counters list, it
    # sets a counter to 0 for the station.
    for x in GraciaStations:        
        if not x['id'] in tablCounter:
            tablCounter[x['id']] = 0
                
        if 0 == x['free_bikes']:            
            tablCounter[x['id']] += 1
        

    for x in neighbourStations:        
        if not x['id'] in tablCounter:
            tablCounter[x['id']] = 0
                
        if 0 == x['free_bikes']:            
            tablCounter[x['id']] += 1



def getStaticFeaturesOfStations(snapshot,volatileFeatures):
    GraciaStations = snapshot['gracia']
    neighbourStations = snapshot['neighbours']       
    
    pandasGraciaStations = pd.DataFrame.from_dict(GraciaStations)
    pandasNeighbourStations = pd.DataFrame.from_dict(neighbourStations)
        
    pandasGraciaStations.drop(volatileFeatures,axis=1)
    pandasNeighbourStations.drop(volatileFeatures,axis=1)
        
    return pandasGraciaStations,pandasNeighbourStations

def getHeatSpots(pandasDF,metrics,m,inverseHeat,heatSpotsList):
    
    '''
    For each station <getHeatSpots> produces a list of spots for a heatmap. 
    Each spot originally represents a segment of time where the station had 0 free_bikes.
    The more time a station has been depleted the hotter it should appear in
    the heatmap.
    
    There is an option for the inverseHeat version of the heatmap. Then, <heat>
    measures availability of bikes in the station.
    
    The amount of time segments of depletion induces the color of the station marker
    in the city map. The color assignation follows the quantiles in the distribution
    of depleted time segments. And such assignation is inmovable whether <inverseHeat>
    is chosen or not.
    '''
    
    stats = metrics['stats']
    counterDF = metrics['pandasTablCounterZeroes'] 
    timeSliceCount = metrics['timeSliceCount']           
    
    for index, row in pandasDF.iterrows():
        heat = counterDF[row.id]
        
        # Basic semantics of <heat> is applied in marker color assignation 
        if heat <= stats['min']:
            st_marker_color='blue'
        elif heat < stats['25%']:
            st_marker_color='green'            
        elif heat < stats['75%']:
            st_marker_color='orange'            
        elif heat < stats['max']:
            st_marker_color='red'
        elif heat == stats['max']:
            st_marker_color='black'
                     
        # Once color marker is assigned, <inverseHeat> may be activated
        if inverseHeat:
            # The parameters for <inverseHeat> were found uppon manual factor exploration
            # based on visual identification for better distinguishable separation area.
            heat = stats['max'] - heat            
            ratioHeat = float(heat) / float(stats['max'])            
            stdev = 2.0 * ratioHeat * ratioHeat * ratioHeat
            scale = 1000.0            
        else:
            ratioHeat = float(heat) / float(stats['max'])
            stdev = 1.0
            scale = 1000.0

        
        folium.CircleMarker([row['latitude'], row['longitude']],
                            radius=15,
                            popup=str(str(row['uid'])+"::"+row['name']),
                            fill_color="#3db7e4", # divvy color
                           ).add_to(m)
        
        folium.Marker([row['latitude'], row['longitude']],                        
                            #popup=str(str(row['name'])+str("::")+str(heat)),
                             popup=str(counterDF[row.id]/timeSliceCount),
                            icon=folium.Icon(color=st_marker_color)
                           ).add_to(m)
         
         
        # produce a list of coordinates for each bike in order to feed the heatmap        
        for s in range(int(100.0*ratioHeat)):        
            disturbLat = ((-stdev+2.0*stdev*rnd.random())/scale)
            disturbLon = ((-stdev+2.0*stdev*rnd.random())/scale)
            heatSpotsList.append([row['latitude']+disturbLat, row['longitude']+disturbLon])
        
        '''
        For each station, each 5' segment with 0 bikes will entail an occurrency 
        in the heatmap giving a random perturbation to original station coordinates.
        '''
        
def heatmap(pandasGraciaStations,pandasNeighbours,metrics,inverseHeat=False):
    ## Must paint in map all x in list <GraciaStations> and all y in y.extra.NearbyStationList
    
    '''
    I got Gracia district coordinates from openstreetmap: 
    https://nominatim.openstreetmap.org/details.php?place_id=198819829
    https://www.openstreetmap.org/relation/3773080#map=14/41.4102/2.1599
    
    map center: 41.41023,2.15087 view on osm.org
    map zoom: 14
    viewbox: 2.08989,41.42632,2.21177,41.39413
    '''    
    
    m = folium.Map([41.41023, 2.15087], zoom_start=14)
      
    # mark each station as a point: put a marker and a pop-up with station name
    # todo : add free_bikes at the pop-up
    zeroBikesList = []    
    #count,mean,std,mini,pct25,pct50,pct75,maxi=counterDF.describe()
    ##stats=counterDF.describe()              
    
    # Get coordinates for occurences to add to heatmap and at markers in map for each Bicing Station
    getHeatSpots(pandasGraciaStations,metrics,m,inverseHeat,zeroBikesList)    
    getHeatSpots(pandasNeighbours,metrics,m,inverseHeat,zeroBikesList)        
    
    ## HEATMAP CALL
    if inverseHeat:
        min_opacity=0.1
        max_val=0.8
    else:
        min_opacity=0.5
        max_val=1.0
    
    
    m.add_child(plugins.HeatMap(zeroBikesList,
                                radius=50,
                                blur=70,
                                min_opacity=min_opacity,
                                max_val=max_val))    
    
    return(m)
    
    #TODO Heatmap radius : add a slider to Jupyter



## GRID##

**def generateGridSpots(pandasGraciaStations,pandasNeighbourStations):**
    
    Given the coordinates of Gracia and Neighbour stations, the method computes the
    enclosing rectangle area. A set of uniform spaced coordinates is generated to identify
    the spots. Each spot will represent a potential home/departure point in the 
    simulation. The method returns a list of pair coordinate and scoring. The scoring
    variable is the future container for the metric Mean Commute Time.
    
    **Issue**: Stations with uid 225 and 405 appear at mislabelled in API as belonging 
    to Gracia district. Just to keep code simple and unchanged compared to first version,
    purging of mislabellings is not managed in the data retrieval methods from first steps
    in the algorithm. The solution is to erase 225 and 405 just when the grid must 
    generated. 
    
    Also, in order to keep simple the code readability, the area which encloses
    the spots is not induced from polygon areas by means of map and hull algorithms.
    
    
**def addGridCommuteTime(dfDots,m):**

    From a dataframe of map spots and its mean commute time, this method paints a spot in the map m.
    

In [18]:
def generateGridSpots(pandasGraciaStations,pandasNeighbourStations):
    
    '''
        Stations with uid 225 and 405 appear at mislabelled in API as belonging 
        to Gracia district.
    '''    
    tabuGraciaUID = [225,405]
    tabuNeighUID = set([])
    for t in tabuGraciaUID:
        tabuNeighUID.update(set(list(pandasGraciaStations[pandasGraciaStations.uid==t].NearbyStationList)[0]))
      
    pandasGraciaStations = pandasGraciaStations[~ pandasGraciaStations.uid.isin(tabuGraciaUID)]
    pandasNeighbourStations = pandasNeighbourStations[~ pandasNeighbourStations.uid.isin(tabuNeighUID)]
    
    spotList = []
    
    maxLat = max(pandasGraciaStations['latitude'].max(),pandasNeighbourStations['latitude'].max())
    minLat = min(pandasGraciaStations['latitude'].min(),pandasNeighbourStations['latitude'].min())
    maxLong = max(pandasGraciaStations['longitude'].max(),pandasNeighbourStations['longitude'].max())
    minLong = min(pandasGraciaStations['longitude'].min(),pandasNeighbourStations['longitude'].min())

    divs = 10.0
    stepLat = (maxLat - minLat)/divs
    stepLong = 0.5*(maxLong - minLong)/divs        
    dotsList = [(x,y,0.0) for x in np.linspace(start=minLat,stop=maxLat,num=divs,endpoint=True,dtype=float) \
                        for y in np.linspace(start=minLong,stop=maxLong,num=2.0*divs,endpoint=True,dtype=float)]

    
    return(dotsList)
    
    
def addGridCommuteTime(dfDots,m):    
    for index, row in dfDots.iterrows():
        folium.CircleMarker([row['latitude'], row['longitude']],
            radius=3,
            popup="{:.2f}".format(row['meanCommuteTime']),
            fill_color="#800054", # divvy color
           ).add_to(m)



## SIMULATION ##


The process fills a distance matrix with the cost in walking time from every spot to every station. The matrix also
contains all the bike time from all the stations to the end point, Catedral Station.

Based on the cost matrix, the routing algorithm choses the optimal route from home to Catedral for every configuration
of generated available bikes across the steps of the simulation. 

The final result of the iterated process of random instantiation and route solving is the mean commute time for each spot in the grid.



### Distance Matrix ###

In [ ]:
##***********************************************************************
def distribSampling(N-2,
                    pandasGraciaStations,
                    pandasNeighbourStations,
                    pandasTablCounterZeroes,
                    timeSliceCount):
                    
    idx = 0
    maskZeros = [-1]*(N-2)
    for index, st in pandasGraciaStations.iterrows():
        probZero = pandasTablCounterZeroes[st.id]/timeSliceCount                
        maskZeros[idx] = rnd.random() < probZero
        # maskZeros[i] = TRUE if # station_i bikes == 0, FALSE otherwise
        idx += 1
    
    for index, st in pandasNeighbourStations.iterrows():
        probZero = pandasTablCounterZeroes[st.id]/timeSliceCount                
        maskZeros[idx] = rnd.random() < probZero
        # maskZeros[i] = TRUE if # station_i bikes == 0, FALSE otherwise
        idx += 1
                
    return maskZeros    
##***********************************************************************
def fillSomeStationDistances(distRow,st,pandasGraciaStations,pandasNeighbourStations,walkSpeed_mps):
    
    pdx = 1
    alfa = (st.latitude,st.longitude)    
    for index, otherSt in pandasGraciaStations.iterrows():        
        stationPos = (otherSt.latitude,otherSt.longitude)
        d = 1000.0 * gpsDistance.manhattanDist(alfa,stationPos)
        timeCost = d / walkSpeed_mps
        distRow[pdx] = timeCost / 60.0 # seconds to minutes
        pdx += 1
    
    for index,otherSt in pandasNeighbourStations.iterrows():
        stationPos = (otherSt.latitude,otherSt.longitude)
        d = 1000.0 * gpsDistance.manhattanDist(alfa,stationPos)
        timeCost = d / walkSpeed_mps
        distRow[pdx] = timeCost / 60.0 # seconds to minutes
        pdx += 1
    
    
##***********************************************************************
def initDistanceCostMatrix(pandasGraciaStations,pandasNeighbourStations,constants,omega):

    ### FILL DISTANCE/COST MATRIX
    # Let's image each dot in dfDots is a potential house/settle point
    # build dist matrix : N = alfa + stations + omega
    N = constants['N']
    # 0: home dot, [1..N-1]: stations, N-1:omega, Catedral station
    dist = [None]*N
    # compute distances from Home to all other stations
    dist[0] = [-1]*N    
    # from Home to Home cost is 0 seconds
    dist[0][0] = 0
    # from Home to Destination at Catedral is not feasible for walking
    dist[0][N-1] = -1
    # ********** compute walking time from Home to Stations in GRACIA
    # !!!!!!!!!! indeed, dist[0][1:N-2] is left unassigned !!!!!!!!!!!.
    # each time we estimate commute time for every spot that slice
    # of the dist matrix shall prepared for this spot. Then simulation
    # process may do its estimations.

    # **********  compute time from Stations to other places in map
    place = 1
    for index, st in pandasGraciaStations.iterrows():
        dist[place] = [-1]*N
        dist[place][0] = -1 # cannot walk back Home
        dist[place][place] = 0.0
        
        fillSomeStationDistances(dist[place],
                             st,
                             pandasGraciaStations,
                             pandasNeighbourStations,
                             constants['walkSpeed_mps'])
    
        # Bike time from station st to Omega
        stationPos = (st.latitude,st.longitude)
        d = 1000.0 * gpsDistance.manhattanDist(stationPos,omega)
        timeCost = d / constants['bikeSpeed_mps']        
        dist[place][N-1] = timeCost / 60.0 # seconds to minutes
        place +=1    

    for index, st in pandasNeighbourStations.iterrows():
        dist[place] = [-1]*N
        dist[place][0] = -1 # cannot walk back Home
        dist[place][place] = 0.0
        
        fillSomeStationDistances(dist[place],
                             st,
                             pandasGraciaStations,
                             pandasNeighbourStations,
                             constants['walkSpeed_mps'])
    
        # Bike time from station st to Omega
        stationPos = (st.latitude,st.longitude)
        d = 1000.0 * gpsDistance.manhattanDist(stationPos,omega)
        timeCost = d / constants['bikeSpeed_mps']        
        dist[place][N-1] = timeCost / 60.0 # seconds to minutes
        place +=1    
    
    # **********  Fill distances from Omega to other places. No outward paths
    dist[N-1] = [-1]*N
    dist[N-1][N-1] = 0.0
    

    return dist                                                   


## SIMULATION core method ##

In [19]:
def SIMUL(pandasGraciaStations, pandasNeighbourStations, metrics, dfDots):
    
    pandasTablCounterZeroes = metrics['pandasTablCounterZeroes']
    timeSliceCount = metrics['timeSliceCount']
    constants = {'walkSpeed_mps':.7, 'bikeSpeed_mps':6.1}    
    N = 1 + len(pandasGraciaStations.index) + len(pandasNeighbourStations.index) +1
    constants['N'] = N       
    
    '''
    DESTINATION = OMEGA NODE:    
        {"empty_slots":5,
        "extra":{"NearbyStationList":[34,35,53,358,380],
            "address":"Catedral",
            "districtCode":"1",
            "status":"OPN",
            "uid":36,
            "zip":"08002"},
        "free_bikes":15,
        "id":"ad600a1e23e604d311151d06490ec7ec",
        "latitude":41.385151,"longitude":2.176804,
        "name":"36 - AV. DE LA CATEDRAL 6",
        "timestamp":"2018-11-25T12:55:40.051000Z"},
    '''
    omega = (41.385151,2.176804)    

    distance = initDistanceCostMatrix(pandasGraciaStations,
                                  pandasNeighbourStations,
                                  constants,
                                  omega)
                                  
    # TODO build cache of graph routing problem   
   
    numSimulationsPerSample = 10

    passedDots=0
    for index, row in dfDots.iterrows():    
        alfaLat,alfaLon,count = row
        alfa = (alfaLat,alfaLon)
        sumCommutes = 0.0
        
        # Assigned times costs in dist matrix adhoc for current case of
        # spot. dist[1:N-2] shall contain distance to all other stations
        pdx = 1    
        for index, st in pandasGraciaStations.iterrows():            
            stationPos = (st.latitude,st.longitude)        
            d = 1000.0 * gpsDistance.manhattanDist(alfa,stationPos)
            timeCost = d / constants['walkSpeed_mps']
            distance[0][pdx] = timeCost / 60.0 # seconds to minutes
            pdx += 1
            
        for index, st in pandasNeighbourStations.iterrows():
            stationPos = (st.latitude,st.longitude)
            d = 1000.0 * gpsDistance.manhattanDist(alfa,stationPos)
            timeCost = d / constants['walkSpeed_mps']
            distance[0][pdx] = timeCost / 60.0 # seconds to minutes
            pdx += 1        
        
        ## PROCEED WITH SIMULATION, iterate and sample through experiments        
        
        for e in range(numSimulationsPerSample):

            ##print("dot %d exp %d" % (passedDots, e))
            passedDots +=1                    
            
            ## Randomly instantiate stations with a number of bikes.
            ## The number of bikes is chosen from the distribution
            ## inside the BSON data. But what matters is if the amount
            ## is zero or not.

            idx = 0
            maskZeros = [-1]*(N-2)
            for index, st in pandasGraciaStations.iterrows():
                probZero = pandasTablCounterZeroes[st.id]/timeSliceCount                
                maskZeros[idx] = rnd.random() < probZero
                # maskZeros[i] = TRUE if # station_i bikes == 0, FALSE otherwise
                idx += 1
            
            for index, st in pandasNeighbourStations.iterrows():
                probZero = pandasTablCounterZeroes[st.id]/timeSliceCount                
                maskZeros[idx] = rnd.random() < probZero
                # maskZeros[i] = TRUE if # station_i bikes == 0, FALSE otherwise
                idx += 1
                        
            
            experDist = copy.deepcopy(distance)
            # Erase routes from alfa to any station with 0 bikes
            # Distance matrix cover entities with idx in [0..N-1]
            # idx=0 : home/alfa; idx=N-1 : omega/Catedral, the rest: stations
            for neigh in range(1,N-1):
                if maskZeros[neigh-1]: # 0 bikes at station 'neigh'
                    experDist[neigh][N-1] = -1
                    # no bikes -> cannot go to omega/Catedral, distance := -1
                
            experDist[0][N-1] = -1 # no walking path from alfa to omega
    
            route, routeTime = DAGShortestWeightedEdgePath.BestRoute(0,N-1,experDist)
            sumCommutes = sumCommutes + routeTime           
            
        row['meanCommuteTime'] = sumCommutes /float(numSimulationsPerSample)
        #ERASE print("commute :"+str(row))
        


In [20]:

    
##***********************************************************************
## Please remember advice from question_0, the leeching process.
## Jupyter notebook will generate results not from a mongoDB query but from 
## a query to mongoDB database dump file.
## All code line preceded by "### MONGO ###" was used in case of direct mongoDB interaction.

### MONGO ### mongoC = MongoClient('mongodb://localhost:27017/')
### MONGO ### dbHosco = mongoC['HOSCO']
### MONGO ### timeBikeAllocation = dbHosco['timeBikeAllocation']

bsonfilePath = 'data/HOSCO/'
bsonfileName = 'timeBikeAllocation.bson'

try:
    with open(bsonfilePath+bsonfileName,'rb') as f:
        timeBikeAllocation_list = bson.decode_all(f.read())
      
except Exception as e:
    print(str(e))


# Get the first snapshot to build the dataframe with station's 
# static features.
### MONGO ### snapshot = timeBikeAllocation.find_one()
snapshot = timeBikeAllocation_list[0]

volatileFeatures = ['free_bikes', 'empty_slots', 'timestamp']
pandasGraciaStations,pandasNeighbourStations = getStaticFeaturesOfStations(snapshot,volatileFeatures)
# remove from pandasNeighbourStations any station contained at pandasGraciaStations 
pandasNeighbourStations = pandasNeighbourStations[~ pandasNeighbourStations.id.isin(pandasGraciaStations.id)]
    
tablCounter = {}
### MONGO ### cursor = timeBikeAllocation.find({})
timeSliceCount = 0
### MONGO ### for snapshot in cursor:
for snapshot in timeBikeAllocation_list:
    countZeroBikeStations(snapshot,tablCounter)
    timeSliceCount += 1

## Prepare data for graphical report;
## paint heatmap from tablCounter

pandasTablCounterZeroes = pd.Series(tablCounter, name='countTimeSliceZeroBikes')
pandasTablCounterZeroes.index.name = 'id'
pandasTablCounterZeroes.reset_index()

# prepare metrics for heatmap 
metrics = { 'timeSliceCount' : timeSliceCount,
         'pandasTablCounterZeroes' : pandasTablCounterZeroes }
stats = pandasTablCounterZeroes.describe()
metrics['stats'] = stats

# HEATMAP
m = heatmap(pandasGraciaStations,
            pandasNeighbourStations,
            metrics,
            inverseHeat=True)

# Grid to set virtual/potential homes   
dotsList = generateGridSpots(pandasGraciaStations,pandasNeighbourStations)

### SIMULATION PART

dfDots = pd.DataFrame(dotsList,columns=['latitude','longitude','meanCommuteTime'])

tsA = datetime.datetime.now().timestamp()
print("Running simulation at "+ str(datetime.datetime.now()))
SIMUL(pandasGraciaStations,
      pandasNeighbourStations,
      metrics,
      dfDots)

tsZ = datetime.datetime.now().timestamp()
print("Elapsed time:"+ "{:.2f}".format(tsZ-tsA) +" seconds.")
addGridCommuteTime(dfDots,m)



/home/atorrano/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
/home/atorrano/.local/lib/python3.5/site-packages/ipykernel_launcher.py:26: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Running simulation at 2018-11-27 15:21:44.168273
Elapsed time:22.26 seconds.


In [21]:
m